<h1 align=center><font size = 6>Applied Data Science Capstone Project</font></h1>

<h1 align=center><font size = 5>By Yash Kamat</font></h1>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy               as np
import pandas              as pd
import matplotlib.pyplot   as plt
import matplotlib.cm       as cm
import matplotlib.colors   as colors
import seaborn             as sns
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import requests
from bs4 import BeautifulSoup

## Retrieving neighborhood data

### Getting the list of neighborhoods

In [4]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Francisco").text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
neighborhoods = []

In [7]:
for row in soup.find_all("div", class_="mw-category")[1].findAll("li"):
    neighborhoods.append(row.text)

In [8]:
neighborhoods

['List of neighborhoods in San Francisco',
 'Alamo Square, San Francisco',
 'Alta Plaza',
 'Anza Vista, San Francisco',
 'Balboa Park, San Francisco',
 'Balboa Terrace, San Francisco',
 'Barbary Coast, San Francisco',
 'Bayview–Hunters Point, San Francisco',
 'Bayview, San Francisco, California',
 'Bernal Heights, San Francisco',
 'Carville, San Francisco',
 'Castro District, San Francisco',
 'Cathedral Hill, San Francisco',
 'Central Embarcadero Piers Historic District',
 'Central Sunset',
 'Chinatown, San Francisco',
 'Civic Center, San Francisco',
 'Cole Valley, San Francisco',
 "Compton's Transgender Cultural District",
 'Corona Heights, San Francisco',
 'Crocker-Amazon, San Francisco',
 'Diamond Heights, San Francisco',
 'Dogpatch, San Francisco',
 'Duboce Park Landmark District',
 'Duboce Triangle, San Francisco',
 'Dumpville',
 'Eureka Valley, San Francisco',
 'Excelsior District, San Francisco',
 'Fillmore District, San Francisco',
 'Financial District, San Francisco',
 "Fisher

In [9]:
neighborhoods = neighborhoods[1:]

In [10]:
# Removing ', Pune' from some neighborhoods to extract just the name of the neighborhood
neighborhoods2 = []

for n in neighborhoods:
    neighborhoods2.append(n.split(',')[0])

del neighborhoods

In [11]:
sf_df = pd.DataFrame({"Neighborhood": neighborhoods2})

In [12]:
sf_df.head(10)

,Neighborhood
0,Alamo Square
1,Alta Plaza
2,Anza Vista
3,Balboa Park
4,Balboa Terrace
5,Barbary Coast
6,Bayview–Hunters Point
7,Bayview
8,Bernal Heights
9,Carville


In [13]:
sf_df.shape

(96, 1)

### Getting neighborhood coordinates

In [14]:
from geopy.geocoders import Nominatim
import geocoder

In [15]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Francisco, CA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [16]:
coords = [ get_latlng(neighborhood) for neighborhood in sf_df["Neighborhood"].tolist() ]

Let's have a look at the first 5 coordinates.

In [17]:
coords[:5]

[[37.77722000000006, -122.43145999999996],
 [37.79101000000003, -122.43929999999995],
 [37.78048000000007, -122.44357999999994],
 [37.72493000000003, -122.44313999999997],
 [37.73180000000008, -122.46739999999994]]

In [18]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:
df = pd.concat([sf_df,df_coords],axis=1)

In [20]:
df.head(10)

,Neighborhood,Latitude,Longitude
0,Alamo Square,37.77722,-122.43146
1,Alta Plaza,37.79101,-122.43930
2,Anza Vista,37.78048,-122.44358
3,Balboa Park,37.72493,-122.44314
4,Balboa Terrace,37.73180,-122.46740
5,Barbary Coast,37.78205,-122.40785
6,Bayview–Hunters Point,37.73431,-122.37943
7,Bayview,37.73345,-122.38998
8,Bernal Heights,37.73903,-122.41621
9,Carville,37.77712,-122.41964


In [21]:
df.shape

(96, 3)

## Visualizing the neighborhoods

In [22]:
import folium

In [23]:
SF = geocoder.arcgis('San Francisco, CA')
lat_lng = SF.latlng
latitude = lat_lng[0]
longitude = lat_lng[1]

In [24]:
sf_map = folium.Map(location=[latitude, longitude], zoom_start=11.5)

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#fda504',
        fill_opacity=0.7,
        parse_html=False).add_to(sf_map)  
    
sf_map

## Exploring neighborhoods

In [25]:
CLIENT_ID = 'LQYMHRPAMLXYDHDQGKOMAMEKNJYG3F2L2MEVDV3O3WLH3NFW'
CLIENT_SECRET = 'DDG4RRGTYT4UI0S5GGABSXKMIOY3CEOWOLNHXQ5XESKHXOND'
VERSION = '20180605'

In [26]:
radius = 500
LIMIT = 100

**Since some of the neighborhoods are close to each other. Hence, to minimalise overlapping, I've set the radius to 500 meters.**

In [27]:
venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude',
                     'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4569, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alamo Square,37.77722,-122.43146,Painted Ladies,37.776120,-122.433389,Historic Site
1,Alamo Square,37.77722,-122.43146,Originals Vinyl,37.775835,-122.431227,Record Shop
2,Alamo Square,37.77722,-122.43146,Alamo Square,37.776045,-122.434363,Park
3,Alamo Square,37.77722,-122.43146,Church of 8 Wheels,37.774733,-122.430862,Roller Rink
4,Alamo Square,37.77722,-122.43146,The Center SF,37.774545,-122.430730,Spiritual Center


**Let's see how many unique venue categories we got and have a look at some of them.**

In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 356 uniques categories.


In [30]:
print(venues_df['VenueCategory'].unique()[:25])

['Historic Site' 'Record Shop' 'Park' 'Roller Rink' 'Spiritual Center'
 'Pakistani Restaurant' 'Dog Run' 'Food Truck' 'German Restaurant'
 'Ice Cream Shop' 'French Restaurant' 'Sushi Restaurant' 'Playground'
 'Spa' 'Sandwich Place' 'Liquor Store' 'Coffee Shop' 'Tennis Court' 'Café'
 'Music Venue' 'Indian Restaurant' 'Bakery' 'Yoga Studio' 'Perfume Shop'
 'Cosmetics Shop']


**Next, let's see how many venues we got per neighborhood.**

In [31]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alamo Square,22,22,22,22,22,22
Alta Plaza,57,57,57,57,57,57
Anza Vista,18,18,18,18,18,18
Balboa Park,8,8,8,8,8,8
Balboa Terrace,10,10,10,10,10,10
Barbary Coast,92,92,92,92,92,92
Bayview,29,29,29,29,29,29
Bayview–Hunters Point,5,5,5,5,5,5
Bernal Heights,36,36,36,36,36,36


San Francisco Residential Park got dropped.

### Analyzing each neighborhood

In [32]:
# One-Hot Encoding
sf_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

sf_onehot['Neighborhood'] = venues_df['Neighborhood'] 
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

In [33]:
sf_onehot.head()

,Zoo Exhibit,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise Ship,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwic

In [34]:
print(sf_onehot.shape)

(4569, 356)


**Next, we'll group the dataframe by neighborhood and see the mean of the frequency of each of venue category.**

In [35]:
sf_grouped = sf_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(sf_grouped.shape)
sf_grouped.head()

(95, 356)


,Neighborhood,Zoo Exhibit,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise Ship,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwic

In [36]:
len(sf_grouped[sf_grouped["Indian Restaurant"] > 0])

31

## Clustering Analysis

In [37]:
sf_ir = sf_grouped[["Neighborhood","Indian Restaurant"]]

In [38]:
from sklearn.cluster import KMeans

In [50]:
kclusters = 3

sf_clustering = sf_ir.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 2, 0, 0, 1, 2], dtype=int32)

In [51]:
sf_merged = sf_ir.copy()

# add clustering labels
sf_merged["Cluster #"] = kmeans.labels_

In [52]:
sf_merged.head(10)

,Neighborhood,Indian Restaurant,Cluster #
0,Alamo Square,0.045455,1
1,Alta Plaza,0.000000,0
2,Anza Vista,0.000000,0
3,Balboa Park,0.000000,0
4,Balboa Terrace,0.000000,0
5,Barbary Coast,0.010870,2
6,Bayview,0.000000,0
7,Bayview–Hunters Point,0.000000,0
8,Bernal Heights,0.027778,1
9,Carville,0.010000,2


In [53]:
df = df[df['Neighborhood']!='San Francisco Residential Park']
sf_merged = sf_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(sf_merged.shape)
sf_merged.head() # check the last columns!

(95, 5)


,Neighborhood,Indian Restaurant,Cluster #,Latitude,Longitude
0,Alamo Square,0.045455,1,37.77722,-122.43146
1,Alta Plaza,0.000000,0,37.79101,-122.43930
2,Anza Vista,0.000000,0,37.78048,-122.44358
3,Balboa Park,0.000000,0,37.72493,-122.44314
4,Balboa Terrace,0.000000,0,37.73180,-122.46740


In [54]:
sf_merged.sort_values(["Cluster #"], inplace=True)
sf_merged

,Neighborhood,Indian Restaurant,Cluster #,Latitude,Longitude
47,Lone Mountain (California),0.000000,0,37.777910,-122.452620
60,North Beach,0.000000,0,37.799090,-122.408450
59,Noe Valley,0.000000,0,37.751600,-122.429080
58,Nob Hill,0.000000,0,37.793360,-122.417870
57,Mount Davidson (California),0.000000,0,37.738260,-122.453300
54,Mission Bay,0.000000,0,37.766900,-122.389790
53,Mid-Market,0.000000,0,37.790997,-122.399210
52,Merced Manor,0.000000,0,37.732440,-122.481450
51,Marina District,0.000000,0,37.803150,-122.436750
50,Manilatown,0.000000,0,37.750900,-122.434240


In [55]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster #']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clustering Analysis

In [56]:
sf_merged.loc[sf_merged['Cluster #'] == 0]

,Neighborhood,Indian Restaurant,Cluster #,Latitude,Longitude
47,Lone Mountain (California),0.0,0,37.777910,-122.452620
60,North Beach,0.0,0,37.799090,-122.408450
59,Noe Valley,0.0,0,37.751600,-122.429080
58,Nob Hill,0.0,0,37.793360,-122.417870
57,Mount Davidson (California),0.0,0,37.738260,-122.453300
54,Mission Bay,0.0,0,37.766900,-122.389790
53,Mid-Market,0.0,0,37.790997,-122.399210
52,Merced Manor,0.0,0,37.732440,-122.481450
51,Marina District,0.0,0,37.803150,-122.436750
50,Manilatown,0.0,0,37.750900,-122.434240


In [57]:
sf_merged.loc[sf_merged['Cluster #'] == 1]

,Neighborhood,Indian Restaurant,Cluster #,Latitude,Longitude
73,Rincon Hill,0.022727,1,37.785750,-122.392445
88,Upper Fillmore,0.024390,1,37.783920,-122.433120
10,Castro District,0.025000,1,37.758490,-122.434770
8,Bernal Heights,0.027778,1,37.739030,-122.416210
79,St. Francis Wood,0.038462,1,37.734650,-122.468030
90,West Portal,0.032258,1,37.739900,-122.467490
74,Russian Hill,0.020833,1,37.802030,-122.419630
91,Western Addition,0.029412,1,37.780950,-122.432220
16,Cole Valley,0.022727,1,37.765320,-122.449870
0,Alamo Square,0.045455,1,37.777220,-122.431460


In [58]:
sf_merged.loc[sf_merged['Cluster #'] == 2]

,Neighborhood,Indian Restaurant,Cluster #,Latitude,Longitude
33,Haight-Ashbury,0.010526,2,37.76993,-122.44692
48,Lower Haight,0.015385,2,37.77273,-122.43310
28,Financial District,0.010309,2,37.79578,-122.40048
5,Barbary Coast,0.010870,2,37.78205,-122.40785
15,Civic Center,0.016129,2,37.77863,-122.41683
82,Tendernob,0.010000,2,37.77712,-122.41964
56,Mission Dolores,0.010000,2,37.76481,-122.42360
24,Dumpville,0.010000,2,37.77712,-122.41964
23,Duboce Triangle,0.010000,2,37.76932,-122.43146
77,South Park,0.010989,2,37.78145,-122.39386
